# Binary operations

In [1]:
import os
import numpy as np
import skimage
#import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
import urllib
#import ipyvolume as ipv
import ipywidgets as ipw
from IPython.display import clear_output

if not os.path.isfile('Samples/confocal2.tif'):
    if not os.path.isdir('Samples'):
        os.mkdir('Samples')
    #url = 'https://cildata.crbs.ucsd.edu/media/images/41678/41678.tif'
    url = 'https://cildata.crbs.ucsd.edu/media/images/41694/41694.tif'
    urllib.request.urlretrieve(url, 'Samples/confocal2.tif');

image = skimage.io.imread('Samples/confocal2.tif')

In [2]:
#image = image[:,:,2]>3
subimage = (image[:,:,2]>3)[350:380,550:580]

In [9]:
truc = np.random.randn(10)

In [12]:
def def_erode(xpos,binary_type,filtersize, threshold, subimage):
    #filtersize = 3
    subimage = (image[:,:,2]>threshold)[360:390,560:590]
    subimage_modif = subimage.copy()
    X,Y = np.meshgrid(np.arange(subimage.shape[0]-filtersize+1),np.arange(subimage.shape[1]-filtersize+1))
    indices = np.stack((np.ravel(X),np.ravel(Y)),axis = 0).T
    #for i in range(0,xpos):
    for j in range(0,xpos):
        subimage2 = subimage[indices[j,0]:indices[j,0]+filtersize,
                             indices[j,1]:indices[j,1]+filtersize]
        filtered = np.zeros(subimage.shape)
        filtered[indices[j,0]:indices[j,0]+filtersize,indices[j,1]:indices[j,1]+filtersize] = 1
        filtered[filtered ==0] = np.nan

        if binary_type == 'erode':
            if np.sum(subimage2)<filtersize**2:
                subimage_modif[indices[j,0],indices[j,1]] = 0
        else:
            if np.sum(subimage2)>0:
                subimage_modif[indices[j,0],indices[j,1]] = 1
    if np.sum(subimage2)<filtersize**2:
        fig = plt.figure(figsize=(10,5))
        ax= plt.subplot(1,2,1)
        plt.imshow(subimage,cmap = 'gray')
        if subimage[indices[j,0],indices[j,1]]==1:
            plt.plot([indices[j,1]],[indices[j,0]],'bo')
            #plt.imshow(filtered,alpha = 0.5, cmap = 'Blues',vmin = 0,vmax = 2.5)
        plt.imshow(filtered,alpha = 0.5, cmap = 'Reds',vmin = 0,vmax = 2.5)
        ax= plt.subplot(1,2,2)
        plt.imshow(subimage_modif,cmap = 'gray')

    else:
        fig = plt.figure(figsize=(10,5))
        ax= plt.subplot(1,2,1)
        plt.imshow(subimage,cmap = 'gray')
        plt.imshow(filtered,alpha = 0.5, cmap = 'Greens',vmin = 0,vmax = 2.5)
        ax= plt.subplot(1,2,2)
        plt.imshow(subimage_modif,cmap = 'gray')


In [21]:
filtersize = ipw.IntSlider(min=3,max=6, value =3)
xpos = ipw.IntSlider(min=1,max=(subimage.shape[0]-3)**2, value = 1,
                                               description = 'Window position')
def update(*args):
    xpos.max = (subimage.shape[0]-filtersize.value+1)**2
    xpos.value = np.min([xpos.max, xpos.value])
filtersize.observe(update)

ipw.interactive(def_erode, xpos = xpos,
                binary_type = ipw.Select(options = ['erode','dilate'], value = 'erode'),
                filtersize = filtersize,
                threshold = ipw.IntSlider(min=0,max=255, value =3),
               subimage = ipw.fixed(subimage))

interactive(children=(IntSlider(value=1, description='Window position', max=784, min=1), Select(description='b…

In [1]:
'''#movie of erosion
X,Y = np.meshgrid(np.arange(subimage.shape[0]-3),np.arange(subimage.shape[1]-3))
indices = np.stack((np.ravel(X),np.ravel(Y)),axis = 0).T
subimage_modif = subimage.copy()
for xpos in range(1,len(indices)):
    for j in range(0,xpos):
        subimage2 = subimage[indices[j,0]:indices[j,0]+4,indices[j,1]:indices[j,1]+4]
        filtered = np.zeros(subimage.shape)
        filtered[indices[j,0]:indices[j,0]+4,indices[j,1]:indices[j,1]+4] = 1
        filtered[filtered ==0] = np.nan

        if np.sum(subimage2)<16:
            subimage_modif[indices[j,0],indices[j,1]] = 0
    if np.sum(subimage2)<16:
        fig = plt.figure(figsize=(10,5))
        ax= plt.subplot(1,2,1)
        plt.imshow(subimage,cmap = 'gray')
        if subimage[indices[j,0],indices[j,1]]==1:
            plt.plot([indices[j,1]],[indices[j,0]],'bo')
            #plt.imshow(filtered,alpha = 0.5, cmap = 'Blues',vmin = 0,vmax = 2.5)
        plt.imshow(filtered,alpha = 0.5, cmap = 'Reds',vmin = 0,vmax = 2.5)
        ax= plt.subplot(1,2,2)
        plt.imshow(subimage_modif,cmap = 'gray')
        fig.tight_layout()
        plt.show()
    else:
        fig = plt.figure(figsize=(10,5))
        ax= plt.subplot(1,2,1)
        plt.imshow(subimage,cmap = 'gray')
        plt.imshow(filtered,alpha = 0.5, cmap = 'Greens',vmin = 0,vmax = 2.5)
        ax= plt.subplot(1,2,2)
        plt.imshow(subimage_modif,cmap = 'gray')
        fig.tight_layout()
        plt.show()
    fig.savefig('binary/binary_'+str(xpos)+'.png')
    clear_output()''';